<a href="https://colab.research.google.com/github/ucfilho/optim_raianars_Insight/blob/master/ANN_gera_matriz_classifica_Insight_CR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# este comando clona o diretorio github ucfilho/ANN no colaboratory research
!git clone https://github.com/RaianaSeixas/ANN #clonar do Github
%cd ANN
#!ls
print("Hello world"+"\n"+"I'm evolving!!!"+"\n")

%matplotlib inline

fatal: destination path 'ANN' already exists and is not an empty directory.
/content/ANN
Hello world
I'm evolving!!!



In [2]:
!ls

000_ANN_Gera_Matriz_out_04_2019.ipynb
000_ANN_parte_001_XY_dados_out_01_2019.ipynb
000_ANN_parte_002_XY_Métricas_D10_OUT_4_2019.ipynb
000_ANN_parte_009_Métricas_PCA_OUT_07_2019.ipynb
000_Gerenciador_DE_var_F_CR_STAT_set_27_2019.ipynb
ANN_2019-10-21_14h30min.csv
ANN_nova_003.csv
drive
imput_ANN_D10_30_out_07_19.zip
imput_ANN_D10_6F.zip
imput_ANN_D10_out_05_19.zip
imput_ANN_schwefel_out_11_19.zip
X_DAT_B_set_17_2019.csv
X_DAT_set_17_2019.csv
XY_D10_30_out_7_2019.csv
XY_D10_out_1_2019.csv
XY_D10_out_5_2019.csv
XY_schwefel_out_11_19.csv
Y_DAT_B_set_17_2019.csv
Y_DAT_set_17_2019.csv


In [0]:
# Functions
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

def tasig(x,deriv=False):
    if(deriv==True):
        return 1-x*x
    return 2/(1+np.exp(-2*x))-1

# purelin function
def purelin(x,deriv=False):
    if(deriv==True):
        return 1
    #N=len(x)
    #for i in range(N):
      #if (x[i]>1):
        #x[i]=1
    return x

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
def ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,X_train,y):
  syn0 = 2*np.random.random((Neuron_input,Neuron_hidden)) - 1
  syn1 = 2*np.random.random((Neuron_hidden,Neuron_output)) - 1
  for j in range(Epoch):
    l0 = X_train
    l1 = tasig(np.dot(l0,syn0))
    l2 = purelin(np.dot(l1,syn1))
    l2_error = y - l2
    l2_delta = l2_error*purelin(l2,deriv=True)
    l1_error = l2_delta.dot(syn1.T)
    l1_delta = l1_error * tasig(l1,deriv=True)
    syn1 += l1.T.dot(l2_delta)*Learning_rate
    syn0 += l0.T.dot(l1_delta)*Learning_rate
  return syn0,syn1



In [0]:
def ANN_ycal(syn0,syn1,X_train):
  l0 = X_train
  l1 = tasig(np.dot(l0,syn0))
  l2 = purelin(np.dot(l1,syn1))
  y_calc=np.reshape(l2,len(l2))
  return y_calc

In [0]:
def Classifica(ycalc):
  n=len(ycalc)
  for i in range(n):
    if(ycalc[i]<0):
      ycalc[i]=-1
    else:
      ycalc[i]=1
  return ycalc

In [0]:
def PrecisaoGlobal(tn, fp, fn, tp):
  PrecGlob=(tn+tp)/(tn+ fp+ fn+ tp)
  return PrecGlob

In [0]:
# leitura dos dados
DAT=pd.read_csv("XY_D10_30_out_7_2019.csv")
#print(DAT.head())
#print(len(DAT))

In [0]:
# separa as colunas de interesse
Index=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)
#print(Dat_Select)
#print(Dat_Select.head())

In [0]:
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [0]:
ColF=DAT_OLD['F']
W=np.array(DAT_OLD['F'])
Wo=np.array(DAT_OLD['Fo'])
for i in range(len(W)):
  W[i]=W[i]-Wo[i]
  if(W[i]<0):
    W[i]=-1
  else:
    W[i]=1
DAT['F'] = W

In [14]:
# obtem as variaveis independentes (treino e teste)
# separa os dados em treino e teste
train=DAT.sample(frac=0.8,random_state=None) 
test=DAT.drop(train.index)
IND=[1,2,3,4,6,7,8,9,10,11,12,13,14]
x_train=train.iloc[:,IND]
x_test=test.iloc[:,IND]
X_OLD=DAT_OLD.iloc[:,IND]
X_NOME=DAT_OLD.columns[IND]
print(X_NOME)


Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')


In [0]:
MELHOR_PRECISAO=0
MAT0=[]
MAT1=[]
MATX=[]
CONFUSAO=[]

In [0]:
# obtem as variaveis dependentes (treino e teste)
#Index=[15] #F
Index=[16] #CR
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
#print(y_train.head())

In [0]:
y_train=np.array(y_train)
Nomes=x_train.columns
x_test=np.array(x_test)
x_train=np.array(x_train)

In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.001
omega=0.9
Epoch=20000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_calc_test=ANN_ycal(syn0,syn1,x_test)


In [19]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)

print("====++++=====+++++=====+++++======+++++=====+++++++")
y_obs_test=np.array(y_obs_test);y_obs_test=Classifica(y_obs_test).ravel();print(y_obs_test)
y_obs_train=np.array(y_obs_train);y_obs_train=Classifica(y_obs_train).ravel();print(y_obs_train)
y_calc_test=np.array(y_calc_test);y_calc_test=Classifica(y_calc_test).ravel();print(y_calc_test)
y_calc_train=np.array(y_calc_train);y_calc_train=Classifica(y_calc_train).ravel();print(y_calc_train)
print("====++++=====+++++=====+++++======+++++=====+++++++")
tn, fp, fn, tp = confusion_matrix(y_calc_train,y_obs_train).ravel()
PrecTreino=PrecisaoGlobal(tn, fp, fn, tp)
print("Treino precisao global=",PrecTreino)
print("Treino tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_train,y_obs_train))

print("====================================")
tn, fp, fn, tp = confusion_matrix(y_calc_test,y_obs_test).ravel()
PrecTeste=PrecisaoGlobal(tn, fp, fn, tp)
print("Teste: precisao global=",PrecTeste)
print("Teste tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_test,y_obs_test))

if(MELHOR_PRECISAO< PrecTeste):
  MELHOR_PRECISAO=PrecTeste
  MAT0=np.copy(syn0)
  MAT1=np.copy(syn1)
  MATX=np.copy(X_OLD)
  CONF_teste=np.copy(confusion_matrix(y_calc_test,y_obs_test))
  CONF_treino=np.copy(confusion_matrix(y_calc_train,y_obs_train))


Epocas= 20000 Learning rate= 0.001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')
====++++=====+++++=====+++++======+++++=====+++++++
[ 1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.  1.  1.]
[ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1.
 -1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1. -1. -1.
  1.  1. -1. -1. -1. -1.  1.]
[ 1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
 -1. -1. -1. -1

In [20]:
print(y_calc_test)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1. -1. -1.
  1.  1. -1. -1. -1. -1.  1.]


In [21]:
print(y_obs_train)
print(y_calc_train)

[ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1.
 -1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.]
[ 1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1.  1.  1. -1.  1. -1.
 -1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.]


In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.0001
omega=0.9
Epoch=100000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_obs_test=y_test
y_calc_test=ANN_ycal(syn0,syn1,x_test)



In [23]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)

y_obs_test=np.array(y_obs_test);y_obs_test=Classifica(y_obs_test).ravel();print(y_obs_test)
y_obs_train=np.array(y_obs_train);y_obs_train=Classifica(y_obs_train).ravel();print(y_obs_train)
y_calc_test=np.array(y_calc_test);y_calc_test=Classifica(y_calc_test).ravel();print(y_calc_test)
y_calc_train=np.array(y_calc_train);y_calc_train=Classifica(y_calc_train).ravel();print(y_calc_train)



tn, fp, fn, tp = confusion_matrix(y_calc_train,y_obs_train).ravel()
PrecTreino=PrecisaoGlobal(tn, fp, fn, tp)
print("Treino precisao global=",PrecTreino)
print("Treino tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_train,y_obs_train))


print("====================================")
tn, fp, fn, tp = confusion_matrix(y_calc_test,y_obs_test).ravel()
PrecTeste=PrecisaoGlobal(tn, fp, fn, tp)
print("Teste: precisao global=",PrecTeste)
print("Teste tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_test,y_obs_test))



if(MELHOR_PRECISAO< PrecTeste):
  MELHOR_PRECISAO=PrecTeste
  MAT0=np.copy(syn0)
  MAT1=np.copy(syn1)
  MATX=np.copy(X_OLD)
  CONF_teste=np.copy(confusion_matrix(y_calc_test,y_obs_test))
  CONF_treino=np.copy(confusion_matrix(y_calc_train,y_obs_train))

Epocas= 100000 Learning rate= 0.0001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'VELOC FOBJ', 'VELOC X', 'Fo', 'CRo', 'fmin',
       'fmax', 'AD_fmin', 'AD_fmax', 'DELTA FOBJ', 'DELTA VX', 'r_fitness'],
      dtype='object')
[ 1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.  1.  1.]
[ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1.
 -1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.]
[ 1.  1. -1. -1. -1. -1.  1. -1. -1.  1. -1.  1.  1.  1. -1. -1. -1. -1.
  1.  1. -1. -1. -1. -1.  1.]
[ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. 

In [0]:
# print(X_NOME)

In [25]:
# ========= Novas mudancas ==================================================
IND=[1,2,6,7]
x_train=train.iloc[:,IND]
x_test=test.iloc[:,IND]
X_OLD=DAT_OLD.iloc[:,IND]
X_NOME=DAT_OLD.columns[IND]
print(X_NOME)
y_train=np.array(y_train)
x_test=np.array(x_test)
x_train=np.array(x_train)

Index(['DI RELATIVO', 'FRAC Its', 'Fo', 'CRo'], dtype='object')


In [0]:
# input dataset
Neuron_input=len(X_NOME)
Neuron_hidden=9
Neuron_output=1
Learning_rate=0.0001
omega=0.9
Epoch=300000
syn0,syn1=ANN(Epoch,Neuron_input,Neuron_hidden,Neuron_output,Learning_rate,x_train,y_train)
y_obs_train=y_train
y_obs_test=y_test
y_calc_train=ANN_ycal(syn0,syn1,x_train)
y_obs_test=y_test
y_calc_test=ANN_ycal(syn0,syn1,x_test)



In [27]:
print("Epocas=",Epoch,"Learning rate=",Learning_rate,"Dados teste matriz confusao:")
print(X_NOME)

y_obs_test=np.array(y_obs_test);y_obs_test=Classifica(y_obs_test).ravel();print(y_obs_test)
y_obs_train=np.array(y_obs_train);y_obs_train=Classifica(y_obs_train).ravel();print(y_obs_train)
y_calc_test=np.array(y_calc_test);y_calc_test=Classifica(y_calc_test).ravel();print(y_calc_test)
y_calc_train=np.array(y_calc_train);y_calc_train=Classifica(y_calc_train).ravel();print(y_calc_train)



tn, fp, fn, tp = confusion_matrix(y_calc_train,y_obs_train).ravel()
PrecTreino=PrecisaoGlobal(tn, fp, fn, tp)
print("Treino precisao global=",PrecTreino)
print("Treino tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_train,y_obs_train))


print("====================================")
tn, fp, fn, tp = confusion_matrix(y_calc_test,y_obs_test).ravel()
PrecTeste=PrecisaoGlobal(tn, fp, fn, tp)
print("Teste: precisao global=",PrecTeste)
print("Teste tn: %d  fp: %d fn: %d tp: %d" % (tn, fp, fn, tp))
print(confusion_matrix(y_calc_test,y_obs_test))



if(MELHOR_PRECISAO< PrecTeste):
  MELHOR_PRECISAO=PrecTeste
  MAT0=np.copy(syn0)
  MAT1=np.copy(syn1)
  MATX=np.copy(X_OLD)
  CONF_teste=np.copy(confusion_matrix(y_calc_test,y_obs_test))
  CONF_treino=np.copy(confusion_matrix(y_calc_train,y_obs_train))



Epocas= 300000 Learning rate= 0.0001 Dados teste matriz confusao:
Index(['DI RELATIVO', 'FRAC Its', 'Fo', 'CRo'], dtype='object')
[ 1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.  1.  1.]
[ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1.
 -1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1.  1.  1. -1.]
[ 1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1.  1.  1. -1.
  1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1. -

In [28]:
print("Melhor precisao=",MELHOR_PRECISAO)
#print(MAT0)
#print(MAT1)
print(pd.DataFrame(MATX))
nl0=MAT0.shape[0];nc0=MAT0.shape[1]
nl1=MAT1.shape[0];nc1=MAT1.shape[1]
print("colunas %d e linhas %d MAT0" % (nc0,nl0))
print("colunas %d e linhas %d MAT1" % (nc1,nl1))
#print(CONF_teste)
#print(CONF_treino)

Melhor precisao= 0.84
            0         1         2         3
0    1.081077  0.200000  0.798558  0.531855
1    1.091500  0.300000  0.618503  0.966139
2    1.107599  0.400000  0.627984  0.944441
3    1.144213  0.500000  0.894793  0.571047
4    1.079718  0.600000  0.901544  0.852554
..        ...       ...       ...       ...
121  0.932264  0.720000  0.981445  0.988007
122  0.928187  0.840000  0.898388  0.103469
123  0.939966  0.960000  0.565138  0.726098
124  0.939669  1.000033  0.943371  0.268703
125  0.939669  1.000033  0.579287  0.936870

[126 rows x 4 columns]
colunas 9 e linhas 4 MAT0
colunas 1 e linhas 9 MAT1


In [29]:
Nomes=['OTIMO','PARTIC_best','Xresp']
print("Melhor precisao=",MELHOR_PRECISAO)
#print(MAT0)
#print(MAT1)
nl0=MAT0.shape[0];nc0=MAT0.shape[1]
nl1=MAT1.shape[0];nc1=MAT1.shape[1]
print("colunas %d e linhas %d MAT0" % (nc0,nl0))
print("colunas %d e linhas %d MAT1" % (nc1,nl1))
#print(CONF_teste)
#print(CONF_treino)
#BUSCA=pd.Series([OTIMO,PARTIC_best,Xresp],index=Nomes )

Melhor precisao= 0.84
colunas 9 e linhas 4 MAT0
colunas 1 e linhas 9 MAT1


In [30]:
dim=np.array([[nl0,nc0],[nl1,nc1]])
Dummies=['hello pandas I am envolving']
Armazenar=['Dummie','MAT0','MAT1','dim','Conf_treino','Conf_teste','precisao']
BUSCA=pd.Series([Dummies,MAT0,MAT1,dim,CONF_treino,CONF_teste,MELHOR_PRECISAO],index=Armazenar )
BUSCA
BUSCA = pd.DataFrame(BUSCA) 
print(BUSCA)


                                                             0
Dummie                           [hello pandas I am envolving]
MAT0         [[0.12131491707709129, -3.2611169080640936, -0...
MAT1         [[0.9629159485674142], [2.4458826413515444], [...
dim                                           [[4, 9], [9, 1]]
Conf_treino                                 [[65, 0], [9, 27]]
Conf_teste                                   [[18, 3], [1, 3]]
precisao                                                  0.84


In [31]:
print(BUSCA)

                                                             0
Dummie                           [hello pandas I am envolving]
MAT0         [[0.12131491707709129, -3.2611169080640936, -0...
MAT1         [[0.9629159485674142], [2.4458826413515444], [...
dim                                           [[4, 9], [9, 1]]
Conf_treino                                 [[65, 0], [9, 27]]
Conf_teste                                   [[18, 3], [1, 3]]
precisao                                                  0.84


In [32]:
cn=nc0+nc1;print(cn)
lmax=max([nl0,nl1]);print(lmax)
print(MAT0.shape);print(MAT1.shape)
syn0=pd.DataFrame(MAT0)
syn1=pd.DataFrame(MAT1)
M=np.zeros((lmax,1))
M[0,0]=nl0;M[1,0]=nc0;M[2,0]=nl1;M[3,0]=nc1;
M=pd.DataFrame(M)
for i in range(nc0):
  M[i+1]=syn0[i]
print(M)
for i in range(nc1):
  M[i+nc0+1]=syn1[i]
print(M)
#M[range(1,nc0+1)]=syn0
Armazenar=['CONF_treino','CONF_teste','MELHOR_PRECISAO']
Complemento=np.zeros(lmax-3)
BUSCA=list(['CONF_treino',CONF_treino,'CONF_teste',CONF_teste,'MELHOR_PRECISAO',MELHOR_PRECISAO] )
M[nc0+nc1+2]=pd.Series(BUSCA)
MATX=pd.DataFrame(MATX)
strings=list(MATX)
MAT_MAX=[]
MAT_MIN=[]
for i in strings:
  max_x=MATX[i].max()
  min_x=MATX[i].min()
  MAT_MAX.append(max_x)
  MAT_MIN.append(min_x)
#print(MAT_MAX)
#print(MAT_MIN)
M[nc0+nc1+3]=pd.Series(MAT_MIN)
M[nc0+nc1+4]=pd.Series(MAT_MAX)
print(M)

10
9
(4, 9)
(9, 1)
     0         1         2         3         4         5         6         7  \
0  4.0  0.121315 -3.261117 -0.242215 -0.329336  1.476191 -1.645264 -2.563130   
1  9.0  0.296760 -1.548832 -1.751770  0.222169  0.147578  1.048306 -1.663161   
2  9.0 -2.146284  2.268599  2.367729 -0.125938 -1.526887  0.718248  1.417951   
3  1.0 -0.106657 -2.646478  0.729148  0.650023  2.523157  1.530936 -2.006753   
4  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
5  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
6  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
7  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
8  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

          8         9  
0  0.766117  2.101636  
1 -2.609547 -3.541875  
2  1.200970  0.718039  
3  0.379368 -0.533301  
4       NaN       NaN  
5       NaN       NaN  
6       NaN 

In [33]:
print(pd.DataFrame(MATX))

            0         1         2         3
0    1.081077  0.200000  0.798558  0.531855
1    1.091500  0.300000  0.618503  0.966139
2    1.107599  0.400000  0.627984  0.944441
3    1.144213  0.500000  0.894793  0.571047
4    1.079718  0.600000  0.901544  0.852554
..        ...       ...       ...       ...
121  0.932264  0.720000  0.981445  0.988007
122  0.928187  0.840000  0.898388  0.103469
123  0.939966  0.960000  0.565138  0.726098
124  0.939669  1.000033  0.943371  0.268703
125  0.939669  1.000033  0.579287  0.936870

[126 rows x 4 columns]


In [34]:
MATX=pd.DataFrame(MATX)
strings=list(MATX)
MAT_MAX=[]
MAT_MIN=[]
for i in strings:
  max_x=MATX[i].max()
  min_x=MATX[i].min()
  MAT_MAX.append(max_x)
  MAT_MIN.append(min_x)
#print(MAT_MAX)
#print(MAT_MIN)
M[nc0+nc1+3]=pd.Series(MAT_MIN)
M[nc0+nc1+4]=pd.Series(MAT_MAX)
print(M)

    0         1         2         3         4         5         6         7   \
0  4.0  0.121315 -3.261117 -0.242215 -0.329336  1.476191 -1.645264 -2.563130   
1  9.0  0.296760 -1.548832 -1.751770  0.222169  0.147578  1.048306 -1.663161   
2  9.0 -2.146284  2.268599  2.367729 -0.125938 -1.526887  0.718248  1.417951   
3  1.0 -0.106657 -2.646478  0.729148  0.650023  2.523157  1.530936 -2.006753   
4  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
5  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
6  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
7  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
8  0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

         8         9         10                  12        13        14  
0  0.766117  2.101636  0.962916         CONF_treino  0.001204  1.378879  
1 -2.609547 -3.541875  2.445883  [[65, 0], [9, 27]]

In [35]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
from datetime import datetime
nowfname=datetime.now().strftime("%Y-%m-%d_%Hh%Mmin")

In [37]:
'''
BUSCA.to_csv(('ANN_'+ nowfname +'.csv'))
print(('ANN_'+ nowfname +'.csv'))
ARQ='ANN_'+ nowfname +'.csv
'''

"\nBUSCA.to_csv(('ANN_'+ nowfname +'.csv'))\nprint(('ANN_'+ nowfname +'.csv'))\nARQ='ANN_'+ nowfname +'.csv\n"

In [38]:

M.to_csv('ANN_nova_003.csv')
!cp ANN_nova_003.csv drive/My\ Drive/imput_ANN/
#!cp  ANN_nova_001.csv drive/My\ Drive/
''''''

''